**1. 데이터 받아오기**

local 테스트이므로 json에서 받아옴(csv는 따옴표 이슈로 제대로 받아와지지 않음)

In [63]:
# -*- coding: utf-8 -*-
import pandas as pd
today = "2023-03-03"
df = pd.read_json(f"{today}.json")


In [64]:
from db import engineconn
from db1 import engineconn1
from db2 import engineconn2
from db_class import Cluster, ClusterNews, Daily, NewsKeyword, Keyword
df.tail()

,news_id,news_title,news_content
14544,865971,"소녀들의 사랑스러운 봄 스타일링 GUESS, 23 S/S POP COLLECTION 출시",토털 라이프 스타일 브랜드 GUESS (이하 '게스')에서 봄을 맞이하여 POP C...
14545,865972,[오늘의 운세] 2023년 03월 03일 띠별 운세,"[쥐띠]참고 기다리면 좋은 일이 생긴다.1948년생, 우연한 기회가 명예를 가져오게..."
14546,865973,나이 들면 왜 '빨간색'이 좋아질까?,◇눈 노화로 빨간색 선호해눈의 노화가 빨간색을 선호하게 한다. 눈의 노화가 진행되면...
14547,865974,"오늘날씨, 아침 출근길 반짝 추위...낮 기온 올라 ‘큰 일교차’",바람도 강하게 불어 체감온도는 더 떨어질 것으로 보인다. 늦은 오후부터 기온이 오르...
14548,865975,비밀 레시피 가득! 맛.잘.알들이 책을 냈다고?,한국의 전통문화를 올바르게 계승하고 현대화해 내일의 유산으로 만들어가는 창조집단 ...


**2. 제목으로 키워드 추출**

bareun 형태소 분석을 이용한다.

2글자 이상이며 숫자로만 이루어지지 않은 일반 명사, 고유 명사만 가져온다.

In [53]:

import bareunpy as brn

API_KEY="koba-BYMLOVY-2QZEFZI-V56WNWI-2LACEXA"
def make_word(text):
    tagger = brn.Tagger(API_KEY, 'localhost')
    res = tagger.tag(text)
    ps = res.pos()
    rs = ''
    for a, b in ps:
        if b[:2] == 'NN' and b != 'NNB' and len(a) > 1 and not a.isnumeric(): rs += ' ' + a
    return rs

df['news_all'] = df['news_title'].str.repeat(3) + df['news_content']
df['keyword'] = df['news_all'].apply(make_word)

**3. TF-IDF 행렬 생성**

만들어진 키워드로 TF - IDF 행렬을 생성한다.

이 때 NaN이 만들어지면 계산이 안되기 때문에 `.dropna()`를 통해 결측치가 나오는 기사는 제거한다.

(NaN이 나오는 이유는 keyword가 생성이 안 된 것으로 보임)

**NEW** 기사별 핵심 키워드를 'best_keyword'로 생성

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vect = CountVectorizer()
document_term_matrix = vect.fit_transform(df['keyword'])

tf = pd.DataFrame(document_term_matrix.toarray(), columns=vect.get_feature_names_out())  
D = len(tf)
ddf = tf.astype(bool).sum(axis=0)
idf = np.log((D+1) / (ddf+1)) + 1

def ky(x):
    ls = []
    for t in x.sort_values(ascending=False)[:11].index.values:
        if t not in ['기자', 'okjebo', '제보', '카톡']: ls.append(t)
    return ls[:10]
df['best_keyword'] = tf.apply(lambda x: ky(x), axis=1)

In [55]:
tfidf = tf * idf                      
tfidf = tfidf / np.linalg.norm(tfidf, axis=1, keepdims=True)

tf_notna = tfidf.dropna()


,0툴즈,10,100선,10인승,11,11번가,120다산콜센터,12인,12인승,12호,...,힘펠,힙노스,힙슐랭,힙스토랑,힙시트,힙입어,힙합,힙해,女性,女聲
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
19163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
19164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.182539,0.0,0.0
19165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


**4. DBSCAN 군집분석**

DBSCAN을 통해 군집분석을 해준다.

거리 eps = 1, min_samples = 5로 잡아서 실행하였다.

In [56]:
from sklearn.cluster import DBSCAN

# epsilon, 최소 샘플 개수 설정
model = DBSCAN(eps=1.1, min_samples=20)
model.fit(tf_notna)
tf_notna['cluster'] = model.labels_


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12896\3758547753.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_notna['cluster'] = model.labels_


,0툴즈,10,100선,10인승,11,11번가,120다산콜센터,12인,12인승,12호,...,힙노스,힙슐랭,힙스토랑,힙시트,힙입어,힙합,힙해,女性,女聲,cluster
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1
19163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2
19164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.182539,0.0,0.0,-1
19165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,-1


제목하고 같이 보여주기
굳이 돌릴 필요 없음

In [57]:
## 제목을 함꼐 가져와서 삽입
tf_notna['title'] = df['news_title'].loc[tf_notna.index]

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12896\3309977815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_notna['title'] = df['news_title'].loc[tf_notna.index]


군집이 어떻게 묶였는지 결과 확인

**5. 군집별 키워드 추출**

각 군집의 키워드를 추출해낸다.

군집을 묶어 다음의 2개의 기준으로 정렬하여 키워드를 추려낸다.

1. 군집 내에서 얼마나 많은 기사에서 나왔나

2. 기사들 내에서 몇 번이나 언급되었나

이를 바탕으로 for문으로 데일리 키워드와 군집별 키워드를 따로 만든다.

`daily_keyword` : 데일리 키워드

`gdf` : 군집 데이터 프레임

In [59]:
import json
import mysql.connector
engine2 = engineconn2()
session2 = engine2.sessionmaker()

kw, dk, nw = [], {}, []
cluster_lst = []
try:
    for i in range(max(tf_notna['cluster'])):
        vect = CountVectorizer()
        document_term_matrix = vect.fit_transform(df['keyword'][tf_notna[tf_notna['cluster'] == i].index])
        tf = pd.DataFrame(document_term_matrix.toarray(), columns=vect.get_feature_names_out())
        D = len(tf)
        ddf = tf.astype(bool).sum(axis=0)
        sdf = ddf.sort_values(ascending=False)
        stf = tf.sum(axis=0).sort_values(ascending=False)
        wd = dict()
        for k in sdf.index.values: wd[k] = [sdf[k], stf[k]]
        group_keyword = sorted(wd.items(), key=lambda x: (-x[1][0], -x[1][1]))[:11]

        c = 1
        kk = []
        for a, b in group_keyword:
            if len(kk) == 10: break
            if a == '기자': continue
            if c: dk[a] = b[0]; c = 0
            kk.append(a)
        kw.append(kk)
        nw.append(list(df.loc[tf_notna[tf_notna['cluster'] == i].index, 'news_id']))
        ####################################
        cluster = Cluster(
            date = str(today),
            top_keyword = str(kk[0])
        )
        session2.add(cluster)
        session2.commit()
        last_cluster_id = cluster.cluster_id
        cluster_lst.append(last_cluster_id)
    ########################################## cluster_news
        for k in list(df.loc[tf_notna[tf_notna['cluster'] == i].index, 'news_id']):
            cluster_news = ClusterNews(
                cluster_id = int(last_cluster_id),
                news_id = int(k)
            )
            session2.add(cluster_news)
        session2.commit()
except:
    session2.rollback()
finally:
    session2.close()

In [60]:
engine = engineconn()
session = engine.sessionmaker()
try:
    with open('C:/Users/SSAFY/Desktop/새 폴더 (2)/keyword.json', 'r') as f:
        keyword1 = json.load(f)
    # 키워드를 하나씩 넣기 db에

    for i in kw:
        for j in i:
            if j in keyword1.keys():
                continue
            keyworddic = Keyword(
                keyword = j
            )
            session.add(keyworddic)
            session.commit()
            keyword_id = keyworddic.keyword_id
            keyword1[str(j)] = keyword_id

    json_string = json.dumps(keyword1)
    
    with open('C:/Users/SSAFY/Desktop/새 폴더 (2)/keyword.json', 'w') as f:
        f.write(json_string)
    gdf = pd.DataFrame({'cluster_id':cluster_lst, 'group_keyword': kw, 'news_index': nw})
    daily_keyword = pd.DataFrame(sorted(dk.items(), key=lambda x: -x[1])[:20], columns = ['keyword', 'count'])
    for idx, row in daily_keyword.iterrows():
        keyword = row['keyword']
        count1 = row['count']
        if not count1:
            count1 = 0
        daily = Daily(
            count = count1,
            daily_date = today,
            keyword_id = keyword1[keyword]
        )
        session.add(daily)
        session.commit()
except Exception as e:
    session.rollback()
finally:
    session.close()

In [61]:
engine1 = engineconn1()
session1 = engine1.sessionmaker()
try:
    for idx, row in df.iterrows():
        news_id_c = row['news_id']
        best_keyword = row['best_keyword']
        for key in best_keyword:
            if key in keyword1.keys():
                news_keyword = NewsKeyword(
                    keyword_id = keyword1[key],
                    news_id = news_id_c
                )
                session1.add(news_keyword)
    session1.commit()
except Exception as e:
    print(e)
    session1.rollback()
finally:
    session1.close()

In [62]:
gdf.to_csv(f"C:/Users/SSAFY/Desktop/새 폴더 (2)/csv/{today}.csv")